In [2]:
# 패키지 임포트

import pandas as pd
import numpy as np
import os
import random
import glob


In [34]:
# 데이터 불러오기

train = pd.read_csv("C:/Users/carzen9/Desktop/김찬우/data/train/train.csv")
train.head()
train.tail()

sub = pd.read_csv("C:/Users/carzen9/Desktop/김찬우/data/sample_submission.csv")
sub.tail()

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7775,80.csv_Day8_23h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
def preprocess_data(data, is_train=True):
    
    temp = data.copy()
    temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]
    
    if is_train == True :
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill') # shift(1)은 해당행을 아래로 1칸내림 즉 -48은 TARGET을 위로 48칸올려 저장하겠다.
        #fillna(method='ffill')이건 0행의 결측치를 1행의 결측치로 채운다는 뜻
        #48은 하루데이터를 의미함 즉 Target1은 다음날의 발전량을 뜻함
        temp['Target2'] = temp['TARGET'].shift(-96).fillna(method='ffill')
        temp = temp.dropna()
        
        return temp.iloc[:-96]
    elif is_train == False:
        temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]
        return temp.iloc[-48:,:] # 뒤에서부터 48번쨰행부터 끝까지 즉 6일차 데이터

df_train = preprocess_data(train)
#df_train.iloc[:48]
    

In [47]:
df_test = []

for i in range(81):
    file_path = "C:/Users/carzen9/Desktop/김찬우/data/test/" + str(i) +".csv"
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    df_test.append(temp) # 열까지 같이들어감, 리스트형태로 들어감

X_test = pd.concat(df_test) # 리스트로 묶여있으니까 concat(데이터프레임합치기)을 이용해 묶어준다.
X_test.shape




(3888, 7)

In [67]:
from sklearn.model_selection import train_test_split
X_train_1, X_valid_1, Y_train_1, Y_valid_1 = train_test_split(df_train.iloc[:,:-2],df_train.iloc[:,-2], test_size=0.3, random_state=1)
X_train_2, X_valid_2, Y_train_2, Y_valid_2 = train_test_split(df_train.iloc[:,:-2],df_train.iloc[:,-1], test_size=0.3, random_state=1)



In [58]:
X_train_1.head()

,Hour,TARGET,DHI,DNI,WS,RH,T
6194,1,0.000000,0,0,3.2,87.17,-3
48824,4,0.000000,0,0,1.6,40.21,0
45215,23,0.000000,0,0,0.5,73.62,16
412,14,39.414376,75,886,1.3,61.04,4
7222,11,47.849793,453,61,2.2,37.61,24


In [64]:
# 퀀타일 지정 후 lgbm임포트
quantiles = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
from lightgbm import LGBMRegressor

In [86]:
# 모델링과 예측 한번에 함수로 만들기

def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) 모델링
    model = LGBMRegressor(objective='quantile', alpha = q,
                                  n_estimators = 10000, bagging_fraction=0.7, learning_rate=0.027,subsample=0.7)
    
    model.fit(X_train, Y_train, eval_metric=["quantile"], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=300,verbose=500)

    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    
    return pred, model



In [87]:
# Target 예측

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):
    
    LGBM_models = []
    LGBM_actual_pred = pd.DataFrame()
    
    for q in quantiles:
        print(q)
        pred, model  = LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1) # 옆으로 붙인다는뜻
        
    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred


        
    
    
    

In [88]:
# Target1
models_1, result_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, X_test)
result_1

0.1
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
[500]	valid_0's quantile: 1.35578
Early stopping, best iteration is:
[327]	valid_0's quantile: 1.34966
0.2
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
[500]	valid_0's quantile: 2.14963
[1000]	valid_0's quantile: 2.146
Early stopping, best iteration is:
[795]	valid_0's quantile: 2.14186
0.3
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
[500]	valid_0's quantile: 2.55476
[1000]	valid_0's quantile: 2.53559
[1500]	valid_0's quantile: 2.53022
[2000]	valid_0's quantile: 2.52519
[2500]	valid_0's quantile: 2.51922
[3000]	valid_0's quantile: 2.51733
[35

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
# Target2
models_2, result_2 = train_data(X_train_2, Y_train_2, X_valid_2, Y_valid_2, X_test)
result_2[:48]

0.1
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
[500]	valid_0's quantile: 1.3967
Early stopping, best iteration is:
[305]	valid_0's quantile: 1.39663
0.2
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
[500]	valid_0's quantile: 2.25272
Early stopping, best iteration is:
[541]	valid_0's quantile: 2.25222
0.3
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 300 rounds
[500]	valid_0's quantile: 2.68575
[1000]	valid_0's quantile: 2.66748
[1500]	valid_0's quantile: 2.64857
[2000]	valid_0's quantile: 2.63815
Early stopping, best iteration is:
[1804]	valid_0's quantile: 2.63467
0.4
[LightGBM] [Warning] bagging_frac

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [78]:
print(result_1.shape,result_2.shape)

(3888, 9) (3888, 9)


In [90]:
sub.loc[sub.id.str.contains("Day7"),"q_0.1":] = result_1.values
sub.loc[sub.id.str.contains("Day8"),"q_0.1":] = result_2.values

sub.to_csv("C:/Users/carzen9/Desktop/김찬우/data/sub.csv", index=False)
